In [1]:
import tensorflow as tf
from python_speech_features import mfcc
from python_speech_features import delta
from python_speech_features import logfbank
import scipy.io.wavfile as wav
import numpy as np
import os

directory_path='/media/saurabh/New Volume/tf_audio_sentiment/data/'
directory = os.fsencode(directory_path)

model_path = "/media/saurabh/New Volume/tf_audio_sentiment/models/model.ckpt"

In [2]:
def audio_to_mfcc(fileurl):
    rate, sig = wav.read(fileurl)
    mfcc_feat = mfcc(sig,rate)
#d_mfcc_feat = delta(mfcc_feat, 2)
#fbank_feat = logfbank(sig,rate)

    return mfcc_feat

In [3]:
max_frames=2000

x = tf.placeholder(tf.float32, [None, 26000]) # mnist data image of shape max_frames*13
y = tf.placeholder(tf.int32, [None]) #  Index of output , 0 = sad , 1 = happy

In [4]:
# define a function, that pads audio so that audio frames = max frames

def pad(input):
   # print(input.shape[0])
     if input.shape[0] < max_frames:
        
        diff = max_frames - input.shape[0]
        
        # pad and return input
        return np.pad(input,((0,diff),(0,0)), mode="constant")
    
     else:
        
        return input
        

In [5]:
# Set model weights
W = tf.Variable(tf.zeros([26000, 2]))
b = tf.Variable(tf.zeros([2]))

#y_one_hot = tf.one_hot(y , 2)

pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax , only needed for inference, while training included in cost func.

#Logits = tf.matmul(x, W) + b

In [6]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# 'Saver' op to save and restore all the variables
saver = tf.train.Saver()



In [11]:
with tf.Session() as sess:
    # Initialize variables
    sess.run(init)

    # Restore model weights from previously saved model
    load_path = saver.restore(sess, model_path)
    print("Model restored from file: %s" % load_path)
    
    accoustic_features = pad(audio_to_mfcc('/media/saurabh/New Volume/tf_audio_sentiment/data/lizzie.wav'))
    accoustic_features=np.reshape(accoustic_features, (1,26000))
    
    print(sess.run([pred], feed_dict={x: accoustic_features}))

INFO:tensorflow:Restoring parameters from /media/saurabh/New Volume/tf_audio_sentiment/models/model.ckpt
Model restored from file: None
[array([[ 0.,  1.]], dtype=float32)]
